## **get_elasticsearch_results**
Cette fonction get_elasticsearch_results(query) prend une requête comme argument, construit une requête Elasticsearch avec multi_match pour chercher dans les champs title et content, et retourne les 3 premiers résultats.

In [1]:
## Install the required packages
## pip install -qU elasticsearch openai
import os
from elasticsearch import Elasticsearch
from openai import OpenAI
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

from langchain.globals import set_debug

set_debug(True)
from langchain_core.messages import HumanMessage, SystemMessage








load_dotenv()

es_client = Elasticsearch(
    "https://371e52c2ddc94eeda8d2dbeb8acc5645.us-central1.gcp.cloud.es.io:443",
    api_key=os.environ["elastic_host"]
)
      
openai_client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
)



# Define the fields to use for each index in Elasticsearch 
index_source_fields = {
    "data_for_rag": [
        "content"
    ],
    "documents": [
        "content"
    ]
}


# Define the Elasticsearch query to retrieve the results
def get_elasticsearch_results(query):
    es_query = {
        "retriever": {
            "rrf": {
                "retrievers": [
                    {
                        "standard": {
                            "query": {
                                "multi_match": {
                                    "query": query,
                                    "fields": [
                                        "title",
                                        "content"
                                    ]
                                }
                            }
                        }
                    },
                    {
                        "standard": {
                            "query": {
                                "multi_match": {
                                    "query": query,
                                    "fields": [
                                        "title",
                                        "content"
                                    ]
                                }
                            }
                        }
                    }
                ]
            }
        },
        "size": 10
    }
    result = es_client.search(index="data_for_rag,documents", body=es_query)
    return result["hits"]["hits"]

def create_openai_prompt(question, results):
    context = ""
    for hit in results:
      source_field = index_source_fields.get(hit["_index"])[0]
      hit_context = hit["_source"][source_field]
      context += f"{hit_context}\n"
    prompt = f"""
  Instructions:
  
  - You are an assistant for question-answering tasks.
  - Answer questions truthfully and factually using only the information presented.
  - If you don't know the answer, just say that you don't know, don't make up an answer! and only use the information provided.
  
  - Use markdown format for code examples.
  - You are correct, factual, precise, and reliable.
  
  Context:
  {context}
  Question: {question}
  Answer:
  """
    return prompt


def generate_openai_completion(user_prompt):
    model = ChatOpenAI(model="gpt-4o")
    chain = user_prompt | model | StrOutputParser()
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an assistant for question-answering tasks."},
            {"role": "user", "content": user_prompt},
        ]
    )
    return response.choices[0].message.content

def generate_openai_completion_v2(user_prompt):
    
    messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="What is the purpose of model regularization?"),
    ]
    
    llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
    
    


if __name__ == "__main__":
    question = "Quelle sont les applications a connaitre et a installer pour m'aider"
    elasticsearch_results = get_elasticsearch_results(question)
    context_prompt = create_openai_prompt(question, elasticsearch_results)
    openai_completion = generate_openai_completion(context_prompt)
    print(openai_completion)

Les applications à connaître et à installer pour vous aider sont :

1. **MyPeugeot**
2. **E-Routes**
3. **eSolutions Charging**
